In [8]:
# 在原版paper的deepfm算法做了一点改进，即fm_component和deep component输出的都不只是一个数了，而是一个响亮
# 大量参照https://github.com/ChenglongChen/tensorflow-DeepFM
# 他对deep fm进行了一点点优化，虽然数据描述不太明白，但是其数据结构设计得确实好，很精简也非常适合这个算法
# deep fm只能处理one-hot类型的，无法直接处理multi-hot特征
import numpy as np
import tensorflow as tf

In [15]:
class EmbeddingLayer(tf.keras.layers.Layer):
    def __init__(self,feat_dim,field_num,emb_dim,*args,**kwargs):
        """

        :param feat_dim: int 每个样本的特征维度，假如每个样本可以被表征为[0,0,0,1,0,1,2]，那feat_dim就应该是7
        :param field_num: int 每个样本的特征的field_num，可以理解为有多少种特征，例如一个样本有性别和年龄两类特征(特征向量可能为[0,1,12])，那field_num就是2
        :param emb_dim: int 对于每个field的嵌入向量维度
        :param args:
        :param kwargs:
        """
        super(EmbeddingLayer,self).__init__(*args,**kwargs)
        self.feat_dim=feat_dim
        self.field_num=field_num
        self.emb_dim=emb_dim
        self.emb_layer=tf.keras.layers.Embedding(input_dim=feat_dim,output_dim=emb_dim)

    def call(self, inputs, **kwargs):

        """

        :param inputs: (feat_indices_batch, feat_value_batch)，分为两部分，你可以把他看做一个手动构造的稀疏矩阵
            例如，如果feat_indices_batch的数据为[[1,5,9],[2,7,8]]；feat_value_batch的数据为[[1,1,2.3],[1,1,0.98]]
            假设第一个样本的特征向量是x，那么x[1]=1, x[5]=1, x[9]=2.3，其余位置取值均为0。
            这样构造是因为每个样本都有field_num个field，每个field的取值只有一种（one-hot或者连续值）
            也就是说每个样本都有field_num个不为0的特征维度。而deep fm算法的嵌入方法是对每一个field嵌入，不管是不是连续值都要嵌入，然后再乘以特征取值
            例如x[9]=2.3，那就要从emb_table里找到第9个emb_vector，然后乘以2.3
        :param kwargs:
        :return:
        """

        # feat_indices_batch: [batch_size, field_num]
        # feat_value_batch: [batch_size, field_num]
        feat_indices_batch,feat_value_batch=inputs
        # 两者形状要相同，并且二者的第二个轴取值维度都是field_num个
        assert feat_indices_batch.shape==feat_value_batch.shape
        assert feat_indices_batch.shape[1:]==[self.field_num]

        emb_vectors=self.emb_layer(feat_indices_batch) # [batch_size, field_num, emb_dim]
        feat_value_batch = tf.expand_dims(feat_value_batch,axis=-1) # [batch_size, field_num, 1]

        # broadcast性质 feat_value_batch会被看做[batch_size, field_num, emb_dim]
        emb_vectors = tf.multiply(emb_vectors,feat_value_batch) # [batch_size, field_num, emb_dim]
        return emb_vectors


feat_indices_arr=[np.random.choice(range(10),size=[1,3],replace=False) for _ in range(9)]
feat_indices_arr=np.concatenate(feat_indices_arr,axis=0).astype(np.float32)
print("feat_indices_arr")
print(feat_indices_arr) #[10,3]

feat_vals_arr=np.concatenate((np.ones(shape=[9,2]),
                              np.random.random(size=[9,1])),axis=1).astype(np.float32)
print("\nfeat_vals_arr")
print(feat_vals_arr) # [10,3]

input_ds=tf.data.Dataset.from_tensor_slices((feat_indices_arr,feat_vals_arr))
batched_ds=input_ds.batch(5)
iterator=iter(batched_ds)
input_batch=next(iterator)


print("\nemb vectors")
emb_layer=EmbeddingLayer(feat_dim=10,field_num=3,emb_dim=4)
emb_vectors=emb_layer(input_batch)
print(emb_vectors)
print(emb_layer.emb_layer.weights)

feat_indices_arr
[[0. 2. 8.]
 [8. 2. 9.]
 [2. 4. 3.]
 [4. 5. 8.]
 [7. 6. 2.]
 [1. 6. 3.]
 [5. 8. 3.]
 [7. 0. 3.]
 [9. 6. 3.]]

feat_vals_arr
[[1.         1.         0.52765805]
 [1.         1.         0.5158172 ]
 [1.         1.         0.9801646 ]
 [1.         1.         0.38281646]
 [1.         1.         0.8272224 ]
 [1.         1.         0.5027076 ]
 [1.         1.         0.3349735 ]
 [1.         1.         0.9097907 ]
 [1.         1.         0.9288939 ]]

emb vectors
tf.Tensor(
[[[-0.00363066 -0.03910058  0.02769772 -0.02341567]
  [ 0.02214828 -0.01374368 -0.0273025   0.04496919]
  [-0.01529532 -0.02506005 -0.00118024  0.02598126]]

 [[-0.02898718 -0.04749297 -0.00223675  0.04923883]
  [ 0.02214828 -0.01374368 -0.0273025   0.04496919]
  [-0.02430143  0.00503824 -0.02555656 -0.01632347]]

 [[ 0.02214828 -0.01374368 -0.0273025   0.04496919]
  [ 0.04276675  0.00664122  0.02008495 -0.0024429 ]
  [ 0.01792878 -0.0323379  -0.01729286 -0.00096627]]

 [[ 0.04276675  0.00664122  0.020084

In [16]:
class FMComponent(tf.keras.layers.Layer):
    def __init__(self,feat_dim,field_num,emb_dim,*args,**kwargs):
        """
        同EmbeddingLayer
        :param feat_dim:
        :param field_num:
        :param emb_dim:
        :param args:
        :param kwargs:
        """
        super(FMComponent,self).__init__(*args,**kwargs)
        self.feat_dim=feat_dim
        self.field_num=field_num
        self.emb_dim=emb_dim

    def build(self, input_shape):
        self.w=tf.Variable(initial_value=tf.random.truncated_normal(shape=[self.feat_dim, self.emb_dim]))

    def call(self, inputs, **kwargs):
        """

        :param inputs: (raw_input_batch,emb_vectors)
            其中raw_input_batch是feat_indices_batch,feat_value_batch 其实就是EmbeddingLayer的输入 用于计算一阶term
            emb_vectors是后者是emb_layer的输出
        :param kwargs:
        :return:
        """
        raw_input_batch,emb_vectors=inputs # emb_vectors: [batch_size, field_num, emb_dim]
        # feat_indices_batch: [batch_size, field_num]
        # feat_value_batch: [batch_size, field_num]
        feat_indices_batch,feat_value_batch=raw_input_batch

        # first order term
        # 使用feat_indices找到embedding_lookup快速找到field_num个权重然后做相乘
        # 例如，如果一个样本x，他在特征维度1、3、5上有取值，那么他的feat_indices=[1,3,5]。那只需要从self.w找到第1、3、5个数就可以了
        # 这样的计算方法更加快速
        # 改进1 一阶输出也是一个向量而非一个标量，这就要求self.w的shape为[self.feat_dim, self.emb_dim]
        # 原本deepFM之中女
        # 一阶做的事情实际上就是，假如一个样本有三个field上的取值为[1,1,1.3]，特征id分别是1，3，5，那么一阶结果就是w1*1+w3*1+w5*1.3
        # 拓展版本就是将w1换成了一个长度为emb_size的向量
        weights=tf.nn.embedding_lookup(params=self.w,ids=tf.cast(feat_indices_batch,tf.int32)) # [batch_size, field_num, self.emb_dim]
        # 需要对feat_value_batch扩充一下，不然无法进行broadcast
        first_order_term = tf.multiply(tf.expand_dims(feat_value_batch,axis=2),weights) # [batch_size, field_num, emb_dim]
        first_order_term = tf.reduce_sum(first_order_term,axis=1) # [batch_size, emb_dim]
        print(first_order_term)


        # second order term
        # 下面这个是fm算法的优化算法 和平方减去平方和
        sum_square=tf.square(tf.reduce_sum(emb_vectors,axis=1)) # [batch_size, emb_dim]
        square_sum=tf.reduce_sum(tf.square(emb_vectors),axis=1) # [batch_size, emb_dim]

        second_order_term=1/2*tf.subtract(sum_square,square_sum) # [batch_size, emb_dim]
        print(second_order_term)

        y_fm=first_order_term+second_order_term
        return y_fm


# 突出一个问题，如果num_fields过多，会导致fm_output的数值膨胀
num_fields=3000
num_features=30000
feat_indices_arr=[np.random.choice(range(1,num_features),size=[1,num_fields],replace=False) for _ in range(9)]
feat_indices_arr=np.concatenate(feat_indices_arr,axis=0).astype(np.float32)
feat_vals_arr=np.concatenate((np.ones(shape=[9,2999]),
                              np.random.random(size=[9,1])),axis=1).astype(np.float32)

input_ds=tf.data.Dataset.from_tensor_slices((feat_indices_arr,feat_vals_arr))
batched_ds=input_ds.batch(5)
iterator=iter(batched_ds)
input_batch=next(iterator)


print("\nemb vectors")
emb_layer=EmbeddingLayer(feat_dim=num_features,field_num=num_fields,emb_dim=4)
emb_vectors=emb_layer(input_batch)
print(emb_vectors)

fm_component=FMComponent(feat_dim=num_features,field_num=num_fields,emb_dim=4)
fm_inputs=(input_batch,emb_vectors)
fm_outputs=fm_component(fm_inputs)
print("\nfm_outputs")
print(fm_outputs)



emb vectors
tf.Tensor(
[[[ 3.9646816e-02  2.3797657e-02 -3.0525697e-02  2.3773909e-03]
  [-2.6029622e-02 -2.9312484e-03  2.2400569e-02 -3.9204825e-02]
  [ 3.4188058e-02 -4.8973858e-02 -1.5361510e-02  2.7281526e-02]
  ...
  [ 1.8965114e-02 -2.6497841e-02  8.8852644e-04 -2.8151739e-02]
  [ 4.1447770e-02  4.8467841e-02  3.9479081e-02  4.1204941e-02]
  [ 1.6248481e-03  1.1679070e-03 -1.9375883e-03  4.2101249e-04]]

 [[-1.1873804e-02 -2.6591385e-02  4.4963505e-02 -2.1384502e-02]
  [ 3.2922849e-03 -1.5337951e-03 -1.8142235e-02  9.0087056e-03]
  [ 3.7219848e-02  2.0950604e-02 -3.7559606e-02 -4.2227637e-02]
  ...
  [ 1.6509663e-02 -2.5190413e-05  4.0651057e-02 -1.7813552e-02]
  [-1.8620729e-02 -4.2375028e-02 -3.2706082e-02  1.8036831e-02]
  [-8.0271885e-03  1.8071879e-02  1.4420348e-02 -1.6763832e-02]]

 [[ 1.3585184e-02  2.6681796e-03 -4.4106841e-02  4.6707753e-02]
  [ 2.8806534e-02 -2.7917279e-02 -2.5102843e-02 -2.8342057e-02]
  [ 3.3310521e-02 -7.3150396e-03 -2.6158893e-02 -2.8007627e-02]


In [4]:
class DeepComponent(tf.keras.layers.Layer):
    """
    深层网络，没啥好说的
    """
    def __init__(self,deep_units_list,*args,**kwargs):
        super(DeepComponent,self).__init__(*args,**kwargs)
        self.deep_layers=list()
        for deep_units in deep_units_list:
            self.deep_layers.append(tf.keras.layers.Dense(units=deep_units,activation=tf.nn.relu))

    def call(self, inputs, **kwargs):
        for deep_layer in self.deep_layers:
            inputs=deep_layer(inputs)
        return inputs

In [6]:

class DeepFM(tf.keras.Model):
    def __init__(self,feat_dim,field_num,emb_dim,deep_units_list,scoring_units=2,*args,**kwargs):
        """
        同EmbeddingLayer描述
        :param feat_dim:
        :param field_num:
        :param emb_dim:
        :param deep_units_list:
        :param scoring_units: int 输出的类别数目，两类问题就是2，三类问题就是3
        :param args:
        :param kwargs:
        """
        super(DeepFM,self).__init__(*args,**kwargs)

        self.emb_layer=EmbeddingLayer(feat_dim=feat_dim,field_num=field_num,emb_dim=emb_dim)
        self.fm_component=FMComponent(feat_dim=feat_dim,field_num=field_num,emb_dim=emb_dim)
        self.deep_component=DeepComponent(deep_units_list=deep_units_list)
        self.scoring_layer=tf.keras.layers.Dense(units=scoring_units,activation=None)

    def call(self, inputs, training=None, mask=None):
        emb_vectors=self.emb_layer(inputs)

        fm_inputs=(inputs,emb_vectors)
        y_fm=self.fm_component(fm_inputs)

        deep_inputs=tf.reshape(emb_vectors,shape=[emb_vectors.shape[0],-1])
        y_deep=self.deep_component(deep_inputs)
        y = self.scoring_layer(tf.concat((y_fm,y_deep),axis=1))

        return y


feat_indices_arr=[np.random.choice(range(10),size=[1,3],replace=False) for _ in range(9)]
feat_indices_arr=np.concatenate(feat_indices_arr,axis=0).astype(np.float32)
feat_vals_arr=np.concatenate((np.ones(shape=[9,2]),
                              np.random.random(size=[9,1])),axis=1).astype(np.float32)

input_ds=tf.data.Dataset.from_tensor_slices((feat_indices_arr,feat_vals_arr))
batched_ds=input_ds.batch(5)
iterator=iter(batched_ds)
input_batch=next(iterator)

deep_fm_model=DeepFM(feat_dim=10,field_num=3000,emb_dim=4,deep_units_list=[10,8])
deep_fm_model(input_batch)



<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[-0.26829186, -0.108273  ],
       [-1.1350449 ,  2.1167572 ],
       [-1.3979748 ,  1.5567256 ],
       [-0.7603167 ,  1.079642  ],
       [-0.16347432, -1.9003195 ]], dtype=float32)>

[[0.67044261 0.75686147]
 [0.14368512 0.98200156]
 [0.7414385  0.31037816]]
tf.Tensor(
[[0.47840872 0.52159128]
 [0.30188948 0.69811052]
 [0.60612684 0.39387316]], shape=(3, 2), dtype=float64)
